# Aplicação 3: Marketing de Produto Bancário

## Dataset e Transformações

In [146]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from feature_engine.discretisation import EqualFrequencyDiscretiser

In [147]:
marketing = pd.read_excel("bank_marketing.xlsx", skiprows=1).iloc[:,1:]

In [148]:
marketing.head()

,Cliente_Comprou_o_Titulo?,Idade,Profissão,Estado_Civil,Formação,Cliente_Devedor?,Saldo_Conta_Corrente,Tem_Hipoteca?,Tem_Emprestimo?,Qte_de_Ligações_Feitas
0,Não,33,Operário,Casado,Ensino Médio,Não,263,Sim,Não,3
1,Não,58,Aposentado,Casado,Ensino Médio,Não,19,Não,Sim,3
2,Sim,83,Aposentado,Casado,Fundamental,Não,425,Não,Não,1
3,Sim,32,Administrador,Solteiro,Ensino Superior,Não,10281,Não,Não,1
4,Sim,31,Administrador,Solteiro,Ensino Médio,Não,360,Sim,Não,1


## Análise

- a) Para o marketing em redes sociais, a plataforma permite a seleção do público baseado em até 2 variáveis dentre as seguintes: idade, profissão, estado civil e escolaridade. Quais variáveis você escolheria com o objetivo de separar as pessoas que compram ou não o produto. (Dica: utilize o IV  para as variáveis acima e avalie os resultados)

In [149]:
df_foco_variaveis = marketing.iloc[:, 0:5]

In [150]:
df_foco_variaveis.head(2)

,Cliente_Comprou_o_Titulo?,Idade,Profissão,Estado_Civil,Formação
0,Não,33,Operário,Casado,Ensino Médio
1,Não,58,Aposentado,Casado,Ensino Médio


In [151]:
efd = EqualFrequencyDiscretiser(q = 5)
efd.fit(df_foco_variaveis[["Idade"]])
df_foco_variaveis["faixas_etarias"] = efd.transform(df_foco_variaveis[["Idade"]])

In [152]:
df_foco_variaveis.groupby("faixas_etarias").agg({"Idade": ["min", "max", "size"]}).reset_index()

faixas_etarias Idade          
                   min max  size
0              0    18  31  2222
1              1    32  36  2008
2              2    37  42  1674
3              3    43  52  1946
4              4    53  95  1834

In [153]:
df_foco_variaveis["faixas_etarias"] = (df_foco_variaveis["faixas_etarias"]
                                       .map({0: "18 a 31",
                                             1: "32 a 36",
                                             2: "37 a 42",
                                             3: "48 a 52",
                                             4: "53 a 95"}))

In [154]:
df_foco_variaveis = df_foco_variaveis.drop(columns={"Idade"})

In [155]:
df_foco_variaveis.head()

,Cliente_Comprou_o_Titulo?,Profissão,Estado_Civil,Formação,faixas_etarias
0,Não,Operário,Casado,Ensino Médio,32 a 36
1,Não,Aposentado,Casado,Ensino Médio,53 a 95
2,Sim,Aposentado,Casado,Fundamental,53 a 95
3,Sim,Administrador,Solteiro,Ensino Superior,32 a 36
4,Sim,Administrador,Solteiro,Ensino Médio,18 a 31


In [156]:
df_foco_variaveis = df_foco_variaveis.value_counts().reset_index()

In [157]:
df_foco_variaveis

,Cliente_Comprou_o_Titulo?,Profissão,Estado_Civil,Formação,faixas_etarias,count
0,Sim,Administrador,Solteiro,Ensino Superior,18 a 31,232
1,Sim,Administrador,Solteiro,Ensino Superior,32 a 36,152
2,Sim,Aposentado,Casado,Ensino Médio,53 a 95,143
3,Não,Administrador,Casado,Ensino Superior,32 a 36,139
4,Sim,Administrador,Casado,Ensino Superior,48 a 52,136
...,...,...,...,...,...,...
742,Não,Técnico,Solteiro,Ensino Superior,53 a 95,1
743,Não,Técnico,Solteiro,Desconhecido,18 a 31,1
744,Não,Técnico,Divorciado,Fundamental,48 a 52,1
745,Não,Técnico,Divorciado,Ensino Superior,18 a 31,1


- VOU PARAR POR AQUI E FAZER POR FEATURE IMPORTANCE

- b) Sobre a linha de comunicação, foi sugerido direcionar ao público com idade entre 30-40, pois, dos 4.778 clientes que compraram o produto, essa foi a faixa com maior quantidade. Você concorda com essa análise? Qual seria a sua sugestão?